# 01-data-export

> Code to put scraped stats into format to match google sheets

The following notebook builds a function to clean, reformat, and export the high school stats scrapped for players in the current week. It will take the input of whatever the current week's excel sheet is and provide an output csv in the same format with scores and stats filled in. This sheet is meant for debugging the data_export function, the final function will be called from data_export.py.

In [1]:
#import relevant packages
import datetime
import importlib

##dependencies for function call
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

##function
from functions.stat_scraper import *

In [2]:
def data_export(uploaded_file):
    
    #read in the excel file of interest, given by the user
    input_table = uploaded_file
    
    #data cleaning of input file
    ##toss observations with no MaxPreps link or game date
    input_part = input_table[input_table['MaxPreps Link'].notna() & input_table['Game Date'].notna()].reset_index(drop = True)
    ##change format of game date to match MaxPreps data
    input_part['Game Date'] = input_part['Game Date'].astype(str).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%m/%d'))
    
    #scrape stats for all players
    input_stats = stat_scraper(input_part['MaxPreps Link'])
    
    #merge stats with this week's data, dropping and renaming columns as necessary
    input_merge = pd.merge(input_part, input_stats, left_on = ['MaxPreps Link', 'Game Date'], right_on = ['Player', 'Date'], how = 'left').drop(['Player', 'Date'], axis=1).rename(columns = {'Result_x': 'Result', 'Result_y': 'Scraped_Result'}).drop(['Game Date', 'MaxPreps Link', 'Opponent'], axis = 1)
    
    #merge back to the main page of data, goal of this is to create output file identical to google sheet format
    final_merge = pd.merge(input_table, input_merge, how = 'left')
    
    #cleaning of final export file
    final_merge[['Result', 'Record', 'Stats/Notes']] = final_merge[['Scraped_Result', 'Current_Record', 'Stats_Condensed']]
    final_merge['Record'] = '(' + final_merge['Record'] + ')'
    export = final_merge.drop(['Scraped_Result', 'Current_Record', 'Stats_Condensed'], axis = 1)
    
    #return the export file
    return(export)
    
#     try:
#         #if all is well, csv will export and return success message
#         export.to_csv('current_week_stats.csv', index = False)
#         return('Scores Successfully Exported!')
#     except:
#         #otherwise, an error message will be returned
#         return('There was an issue exporting scores :(')

In [3]:
test_file = 'week2_test.xlsx'
test_sheet = 'Sheet1'

In [4]:
data_export(test_file, test_sheet)

'Scores Successfully Exported!'

Chris Peal and Marcel Reed have the nonetype issue